In [15]:
import os
import configparser
import logging
import pandas as pd
import oracledb

def setup_logging(config):
    log_file = config.get('logging', 'log_file')
    log_level = config.get('logging', 'log_level')

    log_dir = os.path.dirname(log_file)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    logging.basicConfig(filename=log_file,
                        level=getattr(logging, log_level.upper()),
                        format='%(asctime)s:%(levelname)s:%(message)s')
    console = logging.StreamHandler()
    console.setLevel(logging.ERROR)
    formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

def fetch_data(user, password, dsn, query):
    connection = oracledb.connect(user=user, password=password, dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(query)
    columns = [col[0] for col in cursor.description]
    data = cursor.fetchall()
    cursor.close()
    connection.close()
    df = pd.DataFrame(data, columns=columns)
    return df

def upsert_to_customers(df, app_user, app_password, app_dsn):
    connection = oracledb.connect(user=app_user, password=app_password, dsn=app_dsn)
    cursor = connection.cursor()
    
    for _, row in df.iterrows():
        if pd.isna(row.get('CUSTOMER_ID_app')):
            cursor.execute("""
                INSERT INTO APP.CUSTOMERS (CUSTOMER_ID, CUSTOMER_NAME, CUSTOMER_EMAIL)
                VALUES (:1, :2, :3)
            """, (row['CUSTOMER_ID'], row['CUSTOMER_NAME_app'], row['CUSTOMER_EMAIL_app']))
        else:
            cursor.execute("""
                UPDATE APP.CUSTOMERS
                SET CUSTOMER_NAME = :1, CUSTOMER_EMAIL = :2
                WHERE CUSTOMER_ID = :3
            """, (row['CUSTOMER_NAME_etl'], row['CUSTOMER_EMAIL_etl'], row['CUSTOMER_ID']))
    
    connection.commit()
    connection.close()

def main(env='production'):
    config = configparser.ConfigParser()
    config.read('config/config.ini')

    etl_section = f'{env}_etl'
    app_section = f'{env}_app'

    etl_user = config[etl_section]['username']
    etl_password = config[etl_section]['password']
    etl_dsn = config[etl_section]['dsn']
    app_user = config[app_section]['username']
    app_password = config[app_section]['password']
    app_dsn = config[app_section]['dsn']

    setup_logging(config)
    logging.info(f'Starting the database operations script in {env} environment.')

    etl_query = "SELECT * FROM ETL.S_CUSTOMERS"
    etl_data = fetch_data(etl_user, etl_password, etl_dsn, etl_query)

    app_query = "SELECT * FROM APP.CUSTOMERS"
    app_data = fetch_data(app_user, app_password, app_dsn, app_query)

    df_merged = etl_data.merge(app_data, on="CUSTOMER_ID", how="left", suffixes=('_etl', '_app'))

    upsert_to_customers(df_merged, app_user, app_password, app_dsn)

    logging.info('Data synchronization between ETL.S_CUSTOMERS and APP.CUSTOMERS completed.')

if __name__ == "__main__":
    main()


KeyError: 'CUSTOMER_NAME_app'